In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# TODO RND - ARUSH - find a better way to validate current one just gives a number not very sure how good it is. also training parameter also seems a bit dicy
    # sub-TODO - check if we our current implimentation is indiviually or semi-batch. Basically are we udpating the losses after accumilating a bunch of data or after ever run (shayadse called gradient accumilation)
# TODO RND explore the concept of Progressive Context Increment : first block size is 64, than 128 then 256 and so on, for more efficient trainnig
# TODO RND explore the paper that added another layer to decrease from quadratic training time to linear training time
# TODO PRIORITY-3 - PRANAV - mixture of experts layer implimentation
# TODO PRIORITY-4 - ARUSH - a grapher for the training process validation and test time
# TODO PRIORITY-1 - ARUSH - a function to save the model weights whenever needed
# TODO RND - DEVANSH - Explore Multi-Head Latent Attention (MLA) in DeepSeek-V2 and R1 and if it is relevent to us
# TODO RND Expore various other innovations brought by deepseek ( reference : https://medium.com/@jannadikhemais/the-engineering-innovations-behind-deepseek-how-a-chinese-startup-redefined-ai-efficiency-90ea30788829 )
# TODO RND Understand what are the various tricks used in SmolLM 1 to 3
# TODO PRIORITY-1 - Implimentation of Automatic Mixed Precision(AMP) for 2x to 3x gains in speed and 50% memory usage allowing for larger batch sizes
# TODO PRIORITY-3 - Impliment a tool to check the number of times a particular token exisits in a db and then sort and display a histogram with buckets as to how much of each token from vocab is there - to be able to check if each token as sufficient amount of vocab in the llm.
# TODO PRIORITY-2 - Add a loading bar along with percent, which basically indicates how much dataset is covered.
# TODO RND - A way to analyze the mode, such that to see which model weights have actually been trainned and which are reduntant, basically a way to analyize how useful each parameter is to the model

# arush read RL deepseek



In [ ]:
# from layers.RegexTokenizer import RegexTokenizer
import torch
from torch import nn
from torch.nn import functional as F
from torch.amp import autocast, GradScaler

import time
from transformers import AutoTokenizer

from livelossplot import PlotLosses

from utils import ModelSpecs, TrainingData
from layers.Block import Block
torch.manual_seed(1337)

In [ ]:
# hyperparameters
modelSpecs = ModelSpecs.create('small')
# BATCH_SIZE = 64 # how many independent sequences will we process in parallel?
BATCH_SIZE = 32 # how many independent sequences will we process in parallel?
MAX_ITERS = 2000
LEARNING_RATE = 3e-4
EVAL_INTERVALS = 100
# EVAL_ITERS = 200
EVAL_ITERS = 50
TRAIN_TEST_SPLIT = 0.9
# device = 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Running on {device.capitalize()} device.")


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
stories = TrainingData.TinyStories() # TODO impliment data caching in TrainingDataing Class so that we do not read the txt file again if already read once

In [ ]:
i = 8
text = stories[int(10**6 * 0.730612 * i):int(10**6 * 0.730612 *(i+1) )]
# text = TrainingData.TinyStories()[:int(10**6 * 0.730612 * 20 * 5)]
print(len(text)/10**6,"M")


In [ ]:

data = torch.tensor(tokenizer.encode(text))

n = int(TRAIN_TEST_SPLIT * len(data))
print(len(data)/10**6,"M")

train_data = data[:n]
val_data = data[n:]


In [ ]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - modelSpecs.BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+modelSpecs.BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+modelSpecs.BLOCK_SIZE+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(modelSpecs.VOCAB_SIZE, modelSpecs.N_EMBD)
        self.position_embedding_table = nn.Embedding(modelSpecs.BLOCK_SIZE, modelSpecs.N_EMBD)
        self.blocks = nn.Sequential(*[Block(modelSpecs) for _ in range(modelSpecs.N_LAYER)])
        self.ln_f = nn.LayerNorm(modelSpecs.N_EMBD) # final layer norm
        self.lm_head = nn.Linear(modelSpecs.N_EMBD, modelSpecs.VOCAB_SIZE)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -modelSpecs.BLOCK_SIZE:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [ ]:
@torch.no_grad()
def estimate_loss(model : GPTLanguageModel):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [ ]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scaler = GradScaler(device)

In [ ]:
startTime = time.time()

for iter in range(MAX_ITERS):
    print(f"iter #{iter}")
    torch.cuda.empty_cache()
    # every once in a while evaluate the loss on train and val sets
    if iter % EVAL_INTERVALS == 0 or iter == MAX_ITERS - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time {int((time.time() - startTime)//60)} minutes")
    torch.cuda.empty_cache()
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

endTime = time.time()
print(f"Total Training Time : {int((endTime - startTime)//60)} minutes")

In [ ]:
input_tokens = torch.tensor(tokenizer.encode("there was once a forest with 100s of bushes")).unsqueeze(0).cuda()
# print(input_tokens)

model.eval()

output_tokens = m.generate(input_tokens, max_new_tokens=64)[0]
# print(output_tokens)

model.train()

output : str = tokenizer.decode(output_tokens)


print(output)

In [ ]:
print(int(len(stories)//(10**6 )))

In [ ]:
ITERS = 100
startTime = time.time()
step = BATCH_SIZE * modelSpecs.BLOCK_SIZE * ITERS * 8
liveloss = PlotLosses()

# for i in range(step * 0, len(stories), step):
for i in range(989593600, len(stories), step):
    print("ITER:", i // step, "::::", " STRING INDEX:", i)
    text = stories[i : i + step]
    data = torch.tensor(tokenizer.encode(text))
    n = int(TRAIN_TEST_SPLIT * len(data))
    print("tokens", len(data)/10**6,"M")

    train_data = data[:n]
    val_data = data[n:]





    for iter in range(ITERS):
        print(f"iter #{iter}")
        torch.cuda.empty_cache()
        # every once in a while evaluate the loss on train and val sets
        if iter % EVAL_INTERVALS == 0 or iter == ITERS - 1:
            losses = estimate_loss(model)
            liveloss.update({ 'loss': losses['train'], 'val_loss': losses['val']})
            liveloss.send()
            print("ITER:", i // step, "::::", " STRING INDEX:", i)
            print("tokens", len(data)/10**6,"M")
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time {int((time.time() - startTime)//60)} minutes")
        torch.cuda.empty_cache()
        # sample a batch of data
        xb, yb = get_batch('train')

        # evaluate the loss
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    endTime = time.time()
    print(f"Total Training Time : {int((endTime - startTime)//60)} minutes")

In [ ]:
655,360,000
989,593,600

In [ ]:
input_tokens = torch.tensor(tokenizer.encode(" ")).unsqueeze(0).cuda()
# print(input_tokens)


output_tokens = model.generate(input_tokens, max_new_tokens=100)[0]
# print(output_tokens)

output : str = tokenizer.decode(output_tokens)



print(output)


In [ ]:

def convert_optimizer_state_to_float32(optimizer : torch.optim.AdamW):
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor) and v.dtype == torch.float64:
                state[k] = v.float()


# Saving the model
def save_model(model : nn.Module, optimizer : torch.optim.AdamW, filepath):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, filepath)
    print(f"Model saved to {filepath}")

# Loading the model
def load_model(model : nn.Module, optimizer : torch.optim.AdamW, filepath, device):
    checkpoint = torch.load(filepath, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    convert_optimizer_state_to_float32(optimizer)

    print(f"Model loaded from {filepath}")
    model.to(device)
    return model, optimizer





In [ ]:
# Save after training
save_model(model, optimizer, 'weights/gpt_model_50M.pth') # TODO improve this naming convention

# Later or for inference
# model = GPTLanguageModel()
# optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
# model, optimizer = load_model(model, optimizer, 'gpt_model_checkpoint.pth', device)


In [ ]:
pre_model = GPTLanguageModel()
pre_optimizer = torch.optim.AdamW(pre_model.parameters(), lr=LEARNING_RATE)

loaded_model, loaded_optimizer = load_model(pre_model, pre_optimizer, './weights/gpt_model_50M.pth', device)


In [ ]:
loaded_model = loaded_model.cuda()

In [ ]:
ITERS = 100
startTime = time.time()
step = BATCH_SIZE * modelSpecs.BLOCK_SIZE * ITERS * 8
liveloss = PlotLosses()

# for i in range(step * 0, len(stories), step):
for i in range(989593600, len(stories), step):
    print("ITER:", i // step, "::::", " STRING INDEX:", i)
    text = stories[i : i + step]
    data = torch.tensor(tokenizer.encode(text))
    n = int(TRAIN_TEST_SPLIT * len(data))
    print("tokens", len(data)/10**6,"M")

    train_data = data[:n]
    val_data = data[n:]





    for iter in range(ITERS):
        print(f"iter #{iter}")
        torch.cuda.empty_cache()
        # every once in a while evaluate the loss on train and val sets
        if iter % EVAL_INTERVALS == 0 or iter == ITERS - 1:
            losses = estimate_loss(loaded_model)
            liveloss.update({ 'loss': losses['train'], 'val_loss': losses['val']})
            liveloss.send()
            print("ITER:", i // step, "::::", " STRING INDEX:", i)
            print("tokens", len(data)/10**6,"M")
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time {int((time.time() - startTime)//60)} minutes")
        torch.cuda.empty_cache()
        # sample a batch of data
        xb, yb = get_batch('train')

        # evaluate the loss
        logits, loss = loaded_model(xb, yb)
        loaded_optimizer.zero_grad(set_to_none=True)
        loss.backward()
        loaded_optimizer.step()

    endTime = time.time()
    print(f"Total Training Time : {int((endTime - startTime)//60)} minutes")

In [ ]:
input_tokens = torch.tensor(tokenizer.encode(" ")).unsqueeze(0).cuda()
# print(input_tokens)


output_tokens = loaded_model.generate(input_tokens, max_new_tokens=100)[0]
# print(output_tokens)

output : str = tokenizer.decode(output_tokens)



print(output)
